## 1. Prepare the data

## Mount google drive

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [12]:
!mkdir -p /content/drive/My\ Drive/wikidpedia
!ls -l /content/drive/My\ Drive

total 98
drwx------ 2 root root  4096 May 28 10:02  BloombergAndReuters
drwx------ 2 root root  4096 Jul  6 08:49 'Colab Files'
drwx------ 2 root root  4096 May 19 06:38 'Colab Notebooks'
drwx------ 2 root root  4096 May  4 07:10  comp_cars
drwx------ 2 root root  4096 May 19 06:47  Dmoz
drwx------ 2 root root  4096 May  4 07:09  docs
drwx------ 2 root root  4096 May 20 08:01  Feedpushr
-rw------- 1 root root   151 May 20 09:45  Language-iso2label.gsheet
drwx------ 2 root root  4096 Jun  8 07:38  OVH
-rw------- 1 root root 57587 May 25 10:35 'Self Healing Platform.pptx'
-rw------- 1 root root   151 May 24 09:01 'US Politicians.gsheet'
drwx------ 3 root root  4096 Sep 21 04:49  wikidpedia
drwx------ 2 root root  4096 Apr 27 05:40  yolo-darknet


## Get latest Wikipedia English dump (this will take more than 4 hours)

In [ ]:
# Get latest Wikipedia English dump (this will take more than 4 hours)
! wget -nc -P /content/drive/My\ Drive/wikidpedia "http://download.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2"

--2020-09-21 05:11:08--  http://download.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2
Resolving download.wikimedia.org (download.wikimedia.org)... 103.102.166.224, 2001:df2:e500:ed1a::1
Connecting to download.wikimedia.org (download.wikimedia.org)|103.102.166.224|:80... connected.
HTTP request sent, awaiting response... 301 TLS Redirect
Location: https://download.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2 [following]
--2020-09-21 05:11:08--  https://download.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2
Connecting to download.wikimedia.org (download.wikimedia.org)|103.102.166.224|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2 [following]
--2020-09-21 05:11:08--  https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.

In [ ]:
!pip install wikiextractor

  Created wheel for wikiextractor: filename=wikiextractor-0.1-cp36-none-any.whl size=38821 sha256=b4d07648d7e8fe3f8758440b7e1921403d9a7f4093e9feb48d6f8e4d86446bae
  Stored in directory: /root/.cache/pip/wheels/10/59/30/8e5a627a070165141e54816359f344497523d6273e0ced6aec
Successfully built wikiextractor


In [ ]:
# Extract text using WikiExtractor (this will take about 3 hours)
! python -m wikiextractor.WikiExtractor -o "/content/drive/My\ Drive/wikidpedia/data/wikipedia/" --json \
--filter_disambig_page \
--processes 8 \
"data/enwiki-latest-pages-articles.xml.bz2"

## 2. Index wikipedia

## Install and launch Elasticsearch v7.6.2

In [ ]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2
 
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

## Install Haystack+Farm package

In [ ]:
!pip install https://github.com/deepset-ai/haystack/archive/0.3.0.zip

     | 1.7MB 6.4MB/s
     |████████████████████████████████| 194kB 5.3MB/s 
     |████████████████████████████████| 753.2MB 22kB/s 
     |████████████████████████████████| 778kB 44.3MB/s 
     |████████████████████████████████| 3.0MB 45.7MB/s 
  Created wheel for farm-haystack: filename=farm_haystack-0.3.0-cp36-none-any.whl size=82900 sha256=78ba7c4e4f187247a2768f5866c3bb4a246f7f98cb778c2e25d972da179d4dbc
  Stored in directory: /tmp/pip-ephem-wheel-cache-oxx8r8rm/wheels/03/13/9a/6fcd8ef2d9cdc48ddd2dba885f7707ac178c3caf5affc356cf
Successfully built farm-haystack
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 1.5.1 which is incompatible.
  Found existing installation: torch 1.6.0
    Uninstalling torch-1.6.0:
      Successfully uninstalled torch-1.6.0
  Found existing installation: tokenizers 0.8.1rc2
    Uninstalling tokenizers-0.8.1rc2:
      Successfully uninstalled tokenizers-0.8.1rc2
  Found existing installation: transformers 3.1.0
    Uninstalli

## Load Haystack functions

In [ ]:
HOST = 'localhost' 
PORT = 9200 
INDEX_NAME = 'wikipedia_100_stride_50'

from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from haystack.database.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host=HOST, port=PORT, username="", password="", index=INDEX_NAME)

ModuleNotFoundError: ignored

## clear existing index (optional)

In [ ]:
# clear existing index (optional)
if document_store.client.indices.exists(index=document_store.index):
    print('clear existing inddex')
    document_store.client.indices.delete(index=document_store.index)

## Get all dirs in wikipedia folder

In [ ]:
# Get all dirs in wikipedia folder
from os import listdir
from os.path import isfile, join
import json
from tqdm import tqdm

wikidata_path = "../data/wikipedia"
onlydirs = [f for f in listdir(wikidata_path) if not isfile(join(wikidata_path, f))]

dicts = []
bulk_size = 5000

pbar = tqdm(onlydirs)
for directory in pbar:
    subdirs = [f for f in listdir(join(wikidata_path,directory)) if not isfile(join(wikidata_path,directory))]
    pbar.set_description(f"Processing wikipedia folder {directory}")

    for file in subdirs:
        f = open(join(wikidata_path,directory,file), "r") 
        
        # Each text file contains json structures separated by EOL
        articles = f.read().split("\n")
        
        for article in articles:
            if len(article)==0: continue

            # Article in json format
            json_formatted_article = json.loads(article)

            # Rename keys
            document = {"id": json_formatted_article["id"],
                        "name": json_formatted_article["title"],
                        "url": json_formatted_article["url"], 
                        "text": json_formatted_article["text"]}

            # Add document to bulk
            dicts.append(document)
            
            if len(dicts)>=bulk_size:
                # Index bulk
                try:
                    document_store.write_documents(dicts)
                    dicts.clear()
                except:
                    print("Bulk not indexed")
        
    
if len(dicts) > 0:
    print('final round')
    document_store.write_documents(dicts)
            
print('finished')

## 3. QA on wikipedia

In [ ]:
HOST = 'localhost' 
PORT = 9200 
INDEX_NAME = 'wikipedia_100_stride_50'

from haystack import Finder
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from haystack.database.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host=HOST, port=PORT, username="", password="", index=INDEX_NAME)

09/18/2020 17:47:08 - INFO - elasticsearch -   PUT http://localhost:9200/wikipedia_100_stride_50 [status:400 request:0.004s]


In [ ]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)
READER_DiR = "../models/roberta-base-squad2"
READER_DiR = "../models/electra-base-squad2"
reader = TransformersReader(model=READER_DiR, tokenizer=READER_DiR,  use_gpu=0)

In [ ]:
import logging
logging.disable(logging.WARNING)
finder = Finder(reader, retriever)
prediction = finder.get_answers(question="who is the father of Arya Stark", 
                                top_k_retriever=10, 
                                top_k_reader=3)

In [ ]:
print_answers(prediction)

{   'answers': [   {   'answer': 'Lord Eddard Stark,',
                       'context': 'the Hand to honour her father Lord Eddard '
                                  'Stark, Sansa Stark is enchanted by t',
                       'document_id': 'sBXkiHQBc6r1xEMRXkLv',
                       'meta': {   'id': '1183383',
                                   'name': 'Sansa Stark',
                                   'segment_id': 10,
                                   'url': 'https://en.wikipedia.org/wiki?curid=1183383'},
                       'offset_end': 504,
                       'offset_start': 486,
                       'probability': 0.9655782551513105,
                       'score': None},
                   {   'answer': 'King Robert.',
                       'context': 'ey lacks a father figure from King Robert. '
                                  'While escorting Sansa home, h',
                       'document_id': 'KahJiXQBc6r1xEMR94mw',
                       'meta': {   '

In [ ]:
docs = retriever.retrieve('who is the father of Arya Stark', top_k=3)

In [ ]:
docs[0]

Document(id='sBXkiHQBc6r1xEMRXkLv', text="Sansa are walking through the woods, Joffrey notices Arya sparring with the butcher's boy, Mycah. A fight breaks out and Joffrey is attacked by Nymeria (Arya's direwolf) after Joffrey threatens to hurt Arya. Sansa lies to King Robert about the circumstances of the fight in order to protect both Joffrey and her sister Arya. Since Arya ran off with her wolf to save it, Sansa's wolf is killed instead, estranging the Stark daughters. During the Tourney of the Hand to honour her father Lord Eddard Stark, Sansa Stark is enchanted by the knights performing in the event. At the request", external_source_id=None, question=None, query_score=32.162544, meta={'id': '1183383', 'url': 'https://en.wikipedia.org/wiki?curid=1183383', 'segment_id': 10, 'name': 'Sansa Stark'}, tags=None)